In [ ]:
!pip install torch torchvision torchaudio timm thop torchmetrics tqdm

In [ ]:
import json
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import cv2
import numpy as np
import random
import math
import numbers
import time
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torchvision.ops import DeformConv2d
import matplotlib.pyplot as plt
import timm # PyTorch Image Models
from thop import profile # Tính FLOPs và Params
import pandas as pd 
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score, MulticlassPrecision, MulticlassRecall
from torchmetrics import JaccardIndex
from tqdm import tqdm
import torch.multiprocessing as mp

In [ ]:
# 0. TIỆN ÍCH VÀ HÀM HỖ TRỢ
def set_seed(seed):
    """Cố định seed cho các thư viện để đảm bảo kết quả có thể tái lập."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


class EarlyStopping:
    """Dừng quá trình huấn luyện sớm nếu metric không cải thiện sau một số epoch nhất định."""
    def __init__(self, patience=10, delta=0.001, verbose=True, mode='max', path='checkpoint.pt'):
        """
        Args:
            patience (int): Số epoch chờ đợi sau khi metric ngừng cải thiện.
            delta (float): Mức thay đổi tối thiểu được coi là một sự cải thiện.
            verbose (bool): Nếu True, in thông báo mỗi khi metric cải thiện.
            mode (str): 'min' cho loss, 'max' cho accuracy/dice.
            path (str): Đường dẫn lưu checkpoint của model tốt nhất.
        """
        self.patience = patience
        self.delta = delta
        self.verbose = verbose
        self.mode = mode
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_metric_min = np.Inf

        if self.mode == 'max':
            self.delta *= 1
        else: # min
            self.delta *= -1

    def __call__(self, val_metric, model):
        score = val_metric

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_metric, model)
        elif (score - self.best_score) < self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} / {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_metric, model)
            self.counter = 0

    def save_checkpoint(self, val_metric, model):
        """Lưu model khi metric được cải thiện."""
        if self.verbose:
            print(f'Validation metric improved ({self.val_metric_min:.4f} --> {val_metric:.4f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_metric_min = val_metric

In [ ]:
class Config:
    # --- Data & Path ---
    DATASET_PATH = '/kaggle/input/dataset-busi/Dataset_BUSI_with_GT'
    OUTPUT_MASK_DIR = 'merged_masks'
    CLASS_JSON_PATH = 'class_indices.json'
    MTL_MODEL_CHECKPOINT_PATH = 'best_model.pt'
    SEG_MODEL_CHECKPOINT_PATH = 'best_seg_model.pt'
    CLS_MODEL_CHECKPOINT_PATH = 'best_cls_model.pt'

    # --- Model Architecture ---
    # Lựa chọn: 'multitask', 'sequential' 
    MODEL_TYPE = 'sequential' 
    BACKBONE = 'efficientnet_b4'
    NUM_CLASSES = 3 # benign, malignant, normal
    USE_Deform = True  # True: Dùng Deformable Convolution Block, False: không dùng
    # Lựa chọn classification head: 'capsnet' hoặc 'fc' (Fully Connected)
    CLASSIFICATION_HEAD = 'capsnet' 
    
    # --- Training Parameters ---
    IMG_SIZE = 256
    BATCH_SIZE = 32
    EPOCHS = 100
    LEARNING_RATE = 2e-4
    TEST_RATIO = 0.2
    LR_SCHEDULER = 'plateau' # Lựa chọn: 'plateau', 'cosine'
    EARLY_STOPPING_PATIENCE = 20
    EARLY_STOPPING_DELTA = 0.001 # Thay đổi tối thiểu để coi là cải thiện (cho Dice score)
    
    #--- Ablation Study & Reproducibility ---
    SEED = 42
    NUM_RUNS = 30 # Số lần chạy lại toàn bộ thử nghiệm
    
    # --- Implementation Helper ---
    # True: Chạy thử để debug, False: Chạy trên toàn bộ dataset
    TEST_IMPLEMENTATION = False
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Khởi tạo config
config = Config()

In [ ]:
# PHẦN I: CÁC HÀM XỬ LÝ DATASET
# --- 1. HÀM TẠO DATASET GỐC ---
def make_dataset(root, output_mask_dir='merged_masks'):
    """
    Tạo dataset từ thư mục, gộp các mask và trả về danh sách các đường dẫn đầy đủ.

    Args:
        root (str): Đường dẫn đến thư mục gốc chứa các thư mục con (mỗi thư mục là một lớp).
        output_mask_dir (str, optional): Thư mục lưu các ảnh mask đã gộp.

    Returns:
        list of tuple: Danh sách các mẫu dữ liệu, mỗi phần tử là một tuple:
                       (đường dẫn ảnh gốc, đường dẫn ảnh mask, nhãn dạng số nguyên).
    """
    img_class_names = [cls for cls in os.listdir(root) if os.path.isdir(os.path.join(root, cls))]
    img_class_names.sort()
    class_to_idx = {name: i for i, name in enumerate(img_class_names)}

    with open(config.CLASS_JSON_PATH, 'w') as json_file:
        json.dump(class_to_idx, json_file, indent=4)

    dataset_items = []
    os.makedirs(output_mask_dir, exist_ok=True)

    for class_name, label in class_to_idx.items():
        class_dir = os.path.join(root, class_name)
        original_images = [fname for fname in os.listdir(class_dir) if '_mask' not in fname and fname.endswith(('.png', '.jpg', '.jpeg'))]

        for img_name in original_images:
            img_path = os.path.join(class_dir, img_name)
            base_name = os.path.splitext(img_name)[0]
            gt_files = [gt for gt in os.listdir(class_dir) if gt.startswith(base_name + '_mask')]
            
            if not gt_files:
                continue

            if len(gt_files) > 1:
                merged_mask = None
                for gt_file in gt_files:
                    mask_path = os.path.join(class_dir, gt_file)
                    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                    if merged_mask is None:
                        merged_mask = mask
                    else:
                        merged_mask = cv2.bitwise_or(merged_mask, mask)
                
                merged_mask_name = f"{base_name}_mask_merged.png"
                merged_mask_path = os.path.join(output_mask_dir, merged_mask_name)
                cv2.imwrite(merged_mask_path, merged_mask)
                dataset_items.append((img_path, merged_mask_path, label))
            else:
                mask_path = os.path.join(class_dir, gt_files[0])
                dataset_items.append((img_path, mask_path, label))
    return dataset_items

In [ ]:
# --- 2. CÁC LỚP BIẾN ĐỔI DỮ LIỆU (AUGMENTATION) ---
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms
    def __call__(self, img, mask):
        for t in self.transforms:
            img, mask = t(img, mask)
        return img, mask

class Resize(object):
    def __init__(self, size):
        self.size = (int(size), int(size)) if isinstance(size, numbers.Number) else size
    def __call__(self, img, mask):
        return (cv2.resize(img, self.size, interpolation=cv2.INTER_LINEAR),
                cv2.resize(mask, self.size, interpolation=cv2.INTER_NEAREST))

class RandomHorizontallyFlip(object):
    def __init__(self, p=0.5): self.p = p
    def __call__(self, img, mask):
        return (cv2.flip(img, 1), cv2.flip(mask, 1)) if random.random() < self.p else (img, mask)

class RandomVerticallyFlip(object):
    def __init__(self, p=0.5): self.p = p
    def __call__(self, img, mask):
        return (cv2.flip(img, 0), cv2.flip(mask, 0)) if random.random() < self.p else (img, mask)

class RandomRotate(object):
    def __init__(self, degree): self.degree = degree
    def __call__(self, img, mask):
        rotate_degree = random.uniform(-self.degree, self.degree)
        h, w = img.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, rotate_degree, 1.0)
        img_rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderValue=0)
        mask_rotated = cv2.warpAffine(mask, M, (w, h), flags=cv2.INTER_NEAREST, borderValue=0)
        return img_rotated, mask_rotated

In [ ]:
# --- 3. DATASET CLASS ---
class BUSIDataset(data.Dataset):
    def __init__(self, dataset_items, joint_transform=None, image_transform=None, mask_transform=None):
        """
        Args:
            dataset_items (list): Danh sách từ hàm make_dataset.
            joint_transform (callable, optional): Transform áp dụng đồng thời cho cả ảnh và mask.
            image_transform (callable, optional): Transform chỉ áp dụng cho ảnh (sau joint_transform).
            mask_transform (callable, optional): Transform chỉ áp dụng cho mask (sau joint_transform).
        """
        self.items = dataset_items
        self.joint_transform = joint_transform
        self.image_transform = image_transform
        self.mask_transform = mask_transform

    def __getitem__(self, index):
        img_path, mask_path, label = self.items[index]

        # Đọc ảnh và mask bằng cv2
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Chuyển từ BGR (cv2) sang RGB
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Áp dụng các phép biến đổi hình học (augmentation)
        if self.joint_transform is not None:
            img, mask = self.joint_transform(img, mask)

        # Áp dụng các phép biến đổi riêng lẻ (normalize, to_tensor)
        if self.image_transform is not None:
            img = self.image_transform(img)
        
        if self.mask_transform is not None:
            mask = self.mask_transform(mask)

        return img, mask, torch.tensor(label, dtype=torch.long)

    def __len__(self):
        return len(self.items)

class ClassificationSequentialDataset(data.Dataset):
    """
    Dataset cho giai đoạn 2 của mô hình Sequential.
    Nó nhận vào một mô hình segmentation đã được huấn luyện để tạo ra predicted mask.
    """
    def __init__(self, dataset_items, seg_model, size, image_transform=None):
        self.items = dataset_items
        self.seg_model = seg_model
        self.size = size
        self.image_transform = image_transform
        self.seg_model.eval() # Luôn đặt seg_model ở chế độ eval

    def __getitem__(self, index):
        img_path, _, label = self.items[index] # Không cần mask_path gốc nữa

        # 1. Đọc và chuẩn bị ảnh gốc
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (self.size, self.size), interpolation=cv2.INTER_NEAREST)
        
        # Áp dụng transform cho ảnh gốc để đưa vào seg_model
        # Lưu ý: không dùng augmentation hình học ở đây để đảm bảo sự ổn định
        if self.image_transform:
            img_tensor = self.image_transform(img)

        # 2. Tạo predicted mask từ seg_model
        with torch.no_grad():
            # Thêm chiều batch và chuyển đến device
            img_tensor = img_tensor.unsqueeze(0).to(config.DEVICE)
            predicted_mask = self.seg_model(img_tensor)
            
            # Bỏ chiều batch và chuyển về CPU, binarize mask
            predicted_mask = (predicted_mask.squeeze(0) > 0.5).float()

        # 3. Trả về predicted_mask và label
        return predicted_mask, torch.tensor(label, dtype=torch.long)

    def __len__(self):
        return len(self.items)

In [ ]:
# --- 4. HÀM TIỆN ÍCH: TÍNH MEAN VÀ STD ---
def calculate_mean_std(dataset):
    """Tính toán mean và std của tập dữ liệu hình ảnh."""
    loader = torch.utils.data.DataLoader(dataset, batch_size=10, num_workers=0, shuffle=False)
    
    mean = 0.
    std = 0.
    total_images_count = 0
    
    print("Bắt đầu tính toán mean và std của tập train...")
    for images, _, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_samples

    mean /= total_images_count
    std /= total_images_count
    
    print(f"Tính toán xong!")
    return mean.tolist(), std.tolist()

In [ ]:
# PHẦN II: CÁC KHỐI KIẾN TRÚC (ATTENTION, CAPSULE NETWORK)

def squash(vectors, dim=-1):
    s_squared_norm = (vectors ** 2).sum(dim=dim, keepdim=True)
    scale = s_squared_norm / (1 + s_squared_norm) / torch.sqrt(s_squared_norm + 1e-8)
    return scale * vectors

class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, num_route_nodes, in_channels, out_channels,
                 kernel_size=None, stride=None, num_iterations=3):
        super().__init__()
        self.num_route_nodes = num_route_nodes
        self.num_iterations = num_iterations
        self.num_capsules = num_capsules

        if num_route_nodes != -1:  # DigitCaps
            # Trọng số cho routing, shape (num_capsules, num_route_nodes, in_channels, out_channels)
            self.route_weights = nn.Parameter(
                torch.randn(num_capsules, num_route_nodes, in_channels, out_channels)
            )
        else:  # PrimaryCaps
            self.capsules = nn.ModuleList([
                nn.Conv2d(
                    in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=0
                ) for _ in range(num_capsules)
            ])

    def forward(self, x):
        if self.num_route_nodes != -1: # DigitCaps
            # Input x shape: (batch_size, num_route_nodes, in_channels)
            
            # Sử dụng einsum để tính toán các vector dự đoán (priors) một cách an toàn và rõ ràng
            # 'bni,cnio->bcno' -> priors shape: (batch_size, num_capsules, num_route_nodes, out_channels)
            priors = torch.einsum('bni,cnio->bcno', x, self.route_weights)
            
            # Khởi tạo logits với shape (batch_size, num_capsules, num_route_nodes)
            logits = torch.zeros(*priors.shape[:3], device=x.device)

            for i in range(self.num_iterations):
                # Softmax trên num_route_nodes (dim=2)
                probs = F.softmax(logits, dim=2) 
                
                # Tính tổng có trọng số của các priors để có được vector đầu ra
                # (b,c,n,1) * (b,c,n,o) -> sum(dim=2) -> (b,c,o)
                outputs = squash((probs.unsqueeze(-1) * priors).sum(dim=2))
                
                if i < self.num_iterations - 1:
                    # Cập nhật logits bằng "agreement"
                    # (b,c,n,o) * (b,c,1,o) -> sum(dim=-1) -> (b,c,n)
                    delta = (priors * outputs.unsqueeze(2)).sum(dim=-1)
                    logits = logits + delta
                    
            return outputs # Shape: (batch_size, num_capsules, out_channels)
        
        else: # PrimaryCaps
            # Input x shape: (batch_size, in_channels, height, width)
            outputs = [capsule(x) for capsule in self.capsules]
            outputs = torch.stack(outputs, dim=1)
            batch, n_caps, o_ch, h, w = outputs.size()
            
            # Reshape để có được danh sách các capsules
            # (batch_size, num_capsules, out_channels, h*w) -> (batch_size, h*w, num_capsules, out_channels)
            outputs = outputs.view(batch, n_caps, o_ch, -1).permute(0, 3, 1, 2)
            # (batch_size, h*w*num_capsules, out_channels)
            outputs = outputs.reshape(batch, -1, o_ch)
            return squash(outputs)

class CapsuleNetwork(nn.Module):
    def __init__(self, in_channels, in_hw, num_classes):
        super().__init__()
        self.in_hw = in_hw
        self.in_channels = in_channels
        self.num_classes = num_classes

        self.primary_caps = CapsuleLayer(
            num_capsules=8,
            num_route_nodes=-1,
            in_channels=self.in_channels,
            out_channels=16,
            kernel_size=3,
            stride=3
        )

        kernel, stride = 3, 3
        primary_caps_out_hw = (self.in_hw - kernel) // stride + 1
        num_route_nodes = 8 * primary_caps_out_hw * primary_caps_out_hw

        self.digit_caps = CapsuleLayer(
            num_capsules=self.num_classes,
            num_route_nodes=num_route_nodes,
            in_channels=16, # out_channels của primary_caps
            out_channels=8
        )

    def forward(self, x):
        primary_output = self.primary_caps(x)
        digit_output = self.digit_caps(primary_output)
        v_length = torch.norm(digit_output, dim=-1)
        return v_length

In [ ]:
class DeformableConvBlock(nn.Module):
    """
    Khối Deformable Convolution
    """
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, stride=1):
        super(DeformableConvBlock, self).__init__()
        self.padding = padding
        self.stride = stride
        self.kernel_size = kernel_size

        # Conv layer để sinh ra offset
        self.offset_conv = nn.Conv2d(in_channels, 2 * kernel_size * kernel_size, kernel_size=kernel_size, stride=stride, padding=padding)
        # Deformable Convolution layer
        self.dcn = DeformConv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        offset = self.offset_conv(x)
        x = self.dcn(x, offset)
        x = self.bn(x)
        x = self.relu(x)
        return x

class ConvBlock(nn.Module):
    """
    Khối Convolution tiêu chuẩn.
    """
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

In [ ]:
# =================================================================================
# PHẦN III-A: KIẾN TRÚC CHO MULTITASK MODEL
# =================================================================================
class UNet_MTL(nn.Module):
    def __init__(self, cfg):
        super(UNet_MTL, self).__init__()
        self.cfg = cfg
        self.backbone = timm.create_model(cfg.BACKBONE, pretrained=True, features_only=True)
        
        backbone_channels = self.backbone.feature_info.channels()
        # efficientnet_b4 channels: [24, 32, 56, 160, 448]
        
        conv_block = DeformableConvBlock if cfg.USE_Deform else ConvBlock

        # Số kênh đầu vào = kênh từ upsampling + kênh từ skip connection
        self.upconv4 = nn.ConvTranspose2d(backbone_channels[4], backbone_channels[3], kernel_size=2, stride=2)
        self.dec4 = conv_block(backbone_channels[3] + backbone_channels[3], backbone_channels[3])
        
        self.upconv3 = nn.ConvTranspose2d(backbone_channels[3], backbone_channels[2], kernel_size=2, stride=2)
        self.dec3 = conv_block(backbone_channels[2] + backbone_channels[2], backbone_channels[2])
        
        self.upconv2 = nn.ConvTranspose2d(backbone_channels[2], backbone_channels[1], kernel_size=2, stride=2)
        self.dec2 = conv_block(backbone_channels[1] + backbone_channels[1], backbone_channels[1])
        
        self.upconv1 = nn.ConvTranspose2d(backbone_channels[1], backbone_channels[0], kernel_size=2, stride=2)
        self.dec1 = conv_block(backbone_channels[0] + backbone_channels[0], backbone_channels[0])

        # --- Heads ---
        # 1. Segmentation Head
        self.seg_head = nn.Conv2d(backbone_channels[0], 1, kernel_size=1)

        # 2. Classification Head
        if cfg.CLASSIFICATION_HEAD == 'capsnet':
            bottleneck_hw = cfg.IMG_SIZE // self.backbone.feature_info.reduction()[-1]
            self.cls_head = CapsuleNetwork(
                in_channels=backbone_channels[4], 
                in_hw=bottleneck_hw, 
                num_classes=cfg.NUM_CLASSES
            )
        else: # 'fc'
            self.cls_head = nn.Sequential(
                nn.AdaptiveAvgPool2d(1),
                nn.Flatten(),
                nn.Linear(backbone_channels[4], 512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(512, cfg.NUM_CLASSES)
            )

    def forward(self, x):
        # --- Encoder ---
        features = self.backbone(x)
        enc1_out, enc2_out, enc3_out, enc4_out, bottleneck_out = features

        # --- Classification Branch ---
        cls_output = self.cls_head(bottleneck_out)
        
        # --- Decoder (Segmentation Branch) ---
        d4 = self.upconv4(bottleneck_out)
        d4 = torch.cat([d4, enc4_out], dim=1)
        d4 = self.dec4(d4)
        
        d3 = self.upconv3(d4)
        d3 = torch.cat([d3, enc3_out], dim=1)
        d3 = self.dec3(d3)
        
        d2 = self.upconv2(d3)
        d2 = torch.cat([d2, enc2_out], dim=1)
        d2 = self.dec2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat([d1, enc1_out], dim=1)
        d1 = self.dec1(d1)
        
        # Phóng to feature map cuối cùng về đúng kích thước của ảnh đầu vào (x)
        d1 = F.interpolate(d1, size=x.shape[2:], mode='bilinear', align_corners=False)
        
        # Đầu ra segmentation thường dùng sigmoid để đưa giá trị về [0, 1]
        seg_output = torch.sigmoid(self.seg_head(d1))
        
        return seg_output, cls_output

# =================================================================================
# PHẦN III-B: KIẾN TRÚC CHO SEQUENTIAL MODEL
# =================================================================================

class UNet_Segmentation(nn.Module):
    """
    Kiến trúc U-Net chỉ cho tác vụ Segmentation.
    Về cơ bản là UNet_MTL nhưng đã loại bỏ nhánh classification.
    """
    def __init__(self, cfg):
        super(UNet_Segmentation, self).__init__()
        self.cfg = cfg
        self.backbone = timm.create_model(cfg.BACKBONE, pretrained=True, features_only=True)
        
        backbone_channels = self.backbone.feature_info.channels()
        conv_block = DeformableConvBlock if cfg.USE_Deform else ConvBlock

        # Decoder Path
        self.upconv4 = nn.ConvTranspose2d(backbone_channels[4], backbone_channels[3], kernel_size=2, stride=2)
        self.dec4 = conv_block(backbone_channels[3] + backbone_channels[3], backbone_channels[3])
        
        self.upconv3 = nn.ConvTranspose2d(backbone_channels[3], backbone_channels[2], kernel_size=2, stride=2)
        self.dec3 = conv_block(backbone_channels[2] + backbone_channels[2], backbone_channels[2])
        
        self.upconv2 = nn.ConvTranspose2d(backbone_channels[2], backbone_channels[1], kernel_size=2, stride=2)
        self.dec2 = conv_block(backbone_channels[1] + backbone_channels[1], backbone_channels[1])
        
        self.upconv1 = nn.ConvTranspose2d(backbone_channels[1], backbone_channels[0], kernel_size=2, stride=2)
        self.dec1 = conv_block(backbone_channels[0] + backbone_channels[0], backbone_channels[0])

        # Segmentation Head
        self.seg_head = nn.Conv2d(backbone_channels[0], 1, kernel_size=1)

    def forward(self, x):
        # Encoder
        features = self.backbone(x)
        enc1_out, enc2_out, enc3_out, enc4_out, bottleneck_out = features

        # Decoder
        d4 = self.upconv4(bottleneck_out)
        d4 = torch.cat([d4, enc4_out], dim=1)
        d4 = self.dec4(d4)
        
        d3 = self.upconv3(d4)
        d3 = torch.cat([d3, enc3_out], dim=1)
        d3 = self.dec3(d3)
        
        d2 = self.upconv2(d3)
        d2 = torch.cat([d2, enc2_out], dim=1)
        d2 = self.dec2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat([d1, enc1_out], dim=1)
        d1 = self.dec1(d1)
        
        d1 = F.interpolate(d1, size=x.shape[2:], mode='bilinear', align_corners=False)
        
        seg_output = torch.sigmoid(self.seg_head(d1))
        
        return seg_output

class ClassificationModel(nn.Module):
    """
    Model phân loại nhận đầu vào là một ảnh mask (1 kênh).
    """
    def __init__(self, cfg):
        super(ClassificationModel, self).__init__()
        self.cfg = cfg
        
        # Feature Extractor đơn giản cho ảnh mask
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 256 -> 128
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 128 -> 64
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2) # 64 -> 32
        )
        
        # Tái sử dụng Classification Head đã có
        if cfg.CLASSIFICATION_HEAD == 'capsnet':
            # Kích thước ảnh sau feature extractor: 32x32
            # Số kênh cuối cùng: 64
            self.cls_head = CapsuleNetwork(
                in_channels=64,
                in_hw=32, 
                num_classes=cfg.NUM_CLASSES
            )
        else: # 'fc'
            self.cls_head = nn.Sequential(
                nn.AdaptiveAvgPool2d(1),
                nn.Flatten(),
                nn.Linear(64, 128), # 64 là số kênh từ feature_extractor
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(128, cfg.NUM_CLASSES)
            )

    def forward(self, x):
        # x là ảnh mask có shape (batch, 1, H, W)
        features = self.feature_extractor(x)
        cls_output = self.cls_head(features)
        return cls_output

In [ ]:
# PHẦN IV: LOSS FUNCTIONS
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    def forward(self, pred, target):
        pred = pred.contiguous().view(-1)
        target = target.contiguous().view(-1)
        intersection = (pred * target).sum()
        dice = (2. * intersection + self.smooth) / (pred.sum() + target.sum() + self.smooth)
        return 1 - dice

class MarginLoss(nn.Module):
    def __init__(self, m_pos=0.9, m_neg=0.1, lambda_=0.5):
        super(MarginLoss, self).__init__()
        self.m_pos = m_pos
        self.m_neg = m_neg
        self.lambda_ = lambda_
    def forward(self, y_pred, y_true):
        # y_true is class indices, y_pred is lengths of capsule vectors
        y_true_one_hot = F.one_hot(y_true, num_classes=y_pred.size(1)).float()
        
        pos_loss = F.relu(self.m_pos - y_pred).pow(2)
        neg_loss = F.relu(y_pred - self.m_neg).pow(2)
        
        loss = y_true_one_hot * pos_loss + self.lambda_ * (1 - y_true_one_hot) * neg_loss
        return loss.sum(dim=1).mean()

class UncertaintyWeightedLoss(nn.Module):
    """
    Hàm loss tự động cân bằng trọng số giữa 2 tác vụ dựa trên paper của Kendall et al. 2017)
    """
    def __init__(self, num_tasks=2):
        super(UncertaintyWeightedLoss, self).__init__()
        # Learnable parameters cho mỗi task (log variance)
        self.log_vars = nn.Parameter(torch.zeros(num_tasks))

    def forward(self, loss_seg, loss_cls):
        # Task 1: Segmentation
        precision_seg = torch.exp(-self.log_vars[0])
        weighted_loss_seg = precision_seg * loss_seg + self.log_vars[0]

        # Task 2: Classification
        precision_cls = torch.exp(-self.log_vars[1])
        weighted_loss_cls = precision_cls * loss_cls + self.log_vars[1]
        
        # Trả về tổng loss và các loss riêng lẻ để theo dõi
        return weighted_loss_seg + weighted_loss_cls, loss_seg, loss_cls, self.log_vars.detach().cpu()

In [ ]:
# PHẦN V: TRAINING & EVALUATION LOOP

# ==============================================================================
# PHẦN V-A: HÀM TRAIN/EVAL CHO MÔ HÌNH MTL
# ==============================================================================

def train_one_epoch(model, loader, optimizer, seg_criterion, cls_criterion, uncertainty_loss, epoch):
    model.train()
    
    # Bọc loader bằng tqdm để tạo thanh tiến trình
    # desc sẽ hiển thị mô tả cho thanh tiến trình
    loader_tqdm = tqdm(loader, desc=f"Epoch {epoch+1} [Train]")
    
    total_loss, total_seg_loss, total_cls_loss = 0, 0, 0

    for i, (imgs, masks, labels) in enumerate(loader_tqdm):
        if config.TEST_IMPLEMENTATION and i >= 10:
            break
            
        imgs = imgs.to(config.DEVICE)
        masks = masks.to(config.DEVICE)
        labels = labels.to(config.DEVICE)

        optimizer.zero_grad()

        seg_pred, cls_pred = model(imgs)
        
        loss_s = seg_criterion(seg_pred, masks)
        loss_c = cls_criterion(cls_pred, labels)

        combined_loss, _, _, _ = uncertainty_loss(loss_s, loss_c)
        
        combined_loss.backward()
        optimizer.step()

        # Cập nhật các giá trị loss lên thanh tiến trình
        loader_tqdm.set_postfix(loss=combined_loss.item(), seg_loss=loss_s.item(), cls_loss=loss_c.item())

        total_loss += combined_loss.item()
        total_seg_loss += loss_s.item()
        total_cls_loss += loss_c.item()
        
    num_batches = 10 if config.TEST_IMPLEMENTATION else len(loader)
    return total_loss / num_batches, total_seg_loss / num_batches, total_cls_loss / num_batches

def dice_coefficient(y_true, y_pred):
    smooth = 1.0  # Ngăn chia cho 0
    # Chuyển thành tensor 1-D
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = (y_true_f * y_pred_f).sum()
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)

def evaluate(model, loader, seg_criterion, cls_criterion, config):
    model.eval()
    
    # Khởi tạo metrics từ torchmetrics
    jaccard = JaccardIndex(task="binary").to(config.DEVICE)
    accuracy_metric = MulticlassAccuracy(num_classes=config.NUM_CLASSES, average='macro').to(config.DEVICE)
    f1_metric = MulticlassF1Score(num_classes=config.NUM_CLASSES, average='macro').to(config.DEVICE)
    precision_metric = MulticlassPrecision(num_classes=config.NUM_CLASSES, average='macro').to(config.DEVICE)
    recall_metric = MulticlassRecall(num_classes=config.NUM_CLASSES, average='macro').to(config.DEVICE)

    total_seg_loss, total_cls_loss = 0, 0
    inference_times, all_dice_scores = [], []
    loader_tqdm = tqdm(loader, desc="Evaluating")
    num_batches_to_run = 2 if config.TEST_IMPLEMENTATION else len(loader)

    with torch.no_grad():
        for i, (imgs, masks, labels) in enumerate(loader_tqdm):
            if config.TEST_IMPLEMENTATION and i >= 2: break
            
            imgs, masks, labels = imgs.to(config.DEVICE), masks.to(config.DEVICE), labels.to(config.DEVICE)
            
            start_time = time.time()
            seg_pred, cls_pred = model(imgs)
            inference_times.append(time.time() - start_time)

            loss_s = seg_criterion(seg_pred, masks)
            loss_c = cls_criterion(cls_pred, labels)
            total_seg_loss += loss_s.item()
            total_cls_loss += loss_c.item()
            
            loader_tqdm.set_postfix(seg_loss=loss_s.item(), cls_loss=loss_c.item())

            # Cập nhật metrics
            seg_pred_binary = (seg_pred > 0.5).float()
            jaccard.update(seg_pred_binary, masks.int())
            all_dice_scores.append(dice_coefficient(masks, seg_pred_binary).item())
            
            if config.CLASSIFICATION_HEAD == 'capsnet':
                cls_pred_indices = cls_pred.argmax(dim=1)
            else:
                cls_pred_indices = F.softmax(cls_pred, dim=1).argmax(dim=1)
            
            accuracy_metric.update(cls_pred_indices, labels)
            f1_metric.update(cls_pred_indices, labels)
            precision_metric.update(cls_pred_indices, labels)
            recall_metric.update(cls_pred_indices, labels)
    
    # Tính toán kết quả cuối cùng
    metrics = {
        "seg_loss": total_seg_loss / num_batches_to_run,
        "cls_loss": total_cls_loss / num_batches_to_run,
        "iou": jaccard.compute().item(),
        "dice": sum(all_dice_scores) / len(all_dice_scores) if all_dice_scores else 0.0,
        "accuracy": accuracy_metric.compute().item(),
        "f1": f1_metric.compute().item(),
        "precision": precision_metric.compute().item(),
        "recall": recall_metric.compute().item(),
        "inference_time": sum(inference_times) / len(inference_times) if inference_times else 0.0
    }
    
    # Reset metrics cho lần evaluate tiếp theo
    jaccard.reset(); accuracy_metric.reset(); f1_metric.reset(); precision_metric.reset(); recall_metric.reset()

    return metrics

# ==============================================================================
# PHẦN V-B: CÁC HÀM TRAIN/EVAL CHO MÔ HÌNH SEQUENTIAL
# ==============================================================================

def train_seg_only(model, loader, optimizer, criterion, epoch):
    """Hàm train chỉ cho mô hình segmentation."""
    model.train()
    loader_tqdm = tqdm(loader, desc=f"Epoch {epoch+1} [Train Seg]")
    total_loss = 0
    num_batches = 10 if config.TEST_IMPLEMENTATION else len(loader)

    for i, (imgs, masks, _) in enumerate(loader_tqdm):
        if config.TEST_IMPLEMENTATION and i >= 10: break
        imgs, masks = imgs.to(config.DEVICE), masks.to(config.DEVICE)
        
        optimizer.zero_grad()
        seg_pred = model(imgs)
        loss = criterion(seg_pred, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loader_tqdm.set_postfix(loss=loss.item())
        
    return total_loss / num_batches

def evaluate_seg_only(model, loader, criterion):
    """Hàm đánh giá chỉ cho mô hình segmentation."""
    model.eval()
    total_loss, all_dice_scores = 0, []
    loader_tqdm = tqdm(loader, desc="[Eval Seg]")
    num_batches = 2 if config.TEST_IMPLEMENTATION else len(loader)
    
    with torch.no_grad():
        for i, (imgs, masks, _) in enumerate(loader_tqdm):
            if config.TEST_IMPLEMENTATION and i >= 2: break
            imgs, masks = imgs.to(config.DEVICE), masks.to(config.DEVICE)
            
            seg_pred = model(imgs)
            loss = criterion(seg_pred, masks)
            total_loss += loss.item()
            
            seg_pred_binary = (seg_pred > 0.5).float()
            all_dice_scores.append(dice_coefficient(masks, seg_pred_binary).item())
            
    return total_loss / num_batches, sum(all_dice_scores) / len(all_dice_scores)

def train_cls_only(model, loader, optimizer, criterion, epoch):
    """Hàm train chỉ cho mô hình classification (đầu vào là predicted mask)."""
    model.train()
    loader_tqdm = tqdm(loader, desc=f"Epoch {epoch+1} [Train Cls]")
    total_loss, total_correct, total_samples = 0, 0, 0
    num_batches = 10 if config.TEST_IMPLEMENTATION else len(loader)
    
    for i, (pred_masks, labels) in enumerate(loader_tqdm):
        if config.TEST_IMPLEMENTATION and i >= 10: break
        pred_masks, labels = pred_masks.to(config.DEVICE), labels.to(config.DEVICE)

        optimizer.zero_grad()
        cls_pred = model(pred_masks)
        loss = criterion(cls_pred, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        # Tính accuracy đơn giản để theo dõi
        preds = cls_pred.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)
        loader_tqdm.set_postfix(loss=loss.item(), acc=total_correct/total_samples)

    return total_loss / num_batches

def evaluate_cls_only(model, loader, criterion):
    """Hàm đánh giá chỉ cho mô hình classification."""
    model.eval()
    accuracy_metric = MulticlassAccuracy(num_classes=config.NUM_CLASSES, average='macro').to(config.DEVICE)
    f1_metric = MulticlassF1Score(num_classes=config.NUM_CLASSES, average='macro').to(config.DEVICE)
    total_loss = 0
    num_batches = 2 if config.TEST_IMPLEMENTATION else len(loader)
    loader_tqdm = tqdm(loader, desc="[Eval Cls]")
    with torch.no_grad():
        for i, (pred_masks, labels) in enumerate(loader_tqdm):
            if config.TEST_IMPLEMENTATION and i >= 2: break
            pred_masks, labels = pred_masks.to(config.DEVICE), labels.to(config.DEVICE)
            
            cls_pred = model(pred_masks)
            loss = criterion(cls_pred, labels)
            total_loss += loss.item()

            cls_pred_indices = cls_pred.argmax(dim=1)
            accuracy_metric.update(cls_pred_indices, labels)
            f1_metric.update(cls_pred_indices, labels)

    metrics = {
        "cls_loss": total_loss / num_batches,
        "accuracy": accuracy_metric.compute().item(),
        "f1": f1_metric.compute().item()
    }
    accuracy_metric.reset(); f1_metric.reset()
    return metrics

def evaluate_sequential_pipeline(seg_model, cls_model, loader, seg_criterion):
    """Đánh giá toàn bộ pipeline sequential."""
    seg_model.eval()
    cls_model.eval()
    
    # Khởi tạo metrics từ torchmetrics
    jaccard = JaccardIndex(task="binary").to(config.DEVICE)
    accuracy_metric = MulticlassAccuracy(num_classes=config.NUM_CLASSES, average='macro').to(config.DEVICE)
    f1_metric = MulticlassF1Score(num_classes=config.NUM_CLASSES, average='macro').to(config.DEVICE)
    precision_metric = MulticlassPrecision(num_classes=config.NUM_CLASSES, average='macro').to(config.DEVICE)
    recall_metric = MulticlassRecall(num_classes=config.NUM_CLASSES, average='macro').to(config.DEVICE)

    total_seg_loss = 0
    inference_times, all_dice_scores = [], []
    loader_tqdm = tqdm(loader, desc="[Final Eval Sequential]")
    num_batches_to_run = 2 if config.TEST_IMPLEMENTATION else len(loader)

    with torch.no_grad():
        for i, (imgs, masks, labels) in enumerate(loader_tqdm):
            if config.TEST_IMPLEMENTATION and i >= 2: break
            imgs, masks, labels = imgs.to(config.DEVICE), masks.to(config.DEVICE), labels.to(config.DEVICE)
            
            start_time = time.time()
            # Stage 1: Segmentation
            seg_pred = seg_model(imgs)
            seg_pred_binary_for_cls = (seg_pred > 0.5).float() 
            # Stage 2: Classification
            cls_pred = cls_model(seg_pred_binary_for_cls) # Đầu vào là predicted mask
            inference_times.append(time.time() - start_time)

            # --- Tính metrics ---
            loss_s = seg_criterion(seg_pred, masks)
            total_seg_loss += loss_s.item()
            
            seg_pred_binary = (seg_pred > 0.5).float()
            jaccard.update(seg_pred_binary, masks.int())
            all_dice_scores.append(dice_coefficient(masks, seg_pred_binary).item())
            
            cls_pred_indices = cls_pred.argmax(dim=1)
            accuracy_metric.update(cls_pred_indices, labels)
            f1_metric.update(cls_pred_indices, labels)
            precision_metric.update(cls_pred_indices, labels)
            recall_metric.update(cls_pred_indices, labels)

    metrics = {
        "seg_loss": total_seg_loss / num_batches_to_run,
        "cls_loss": -1, # Không có cls_loss ở đây
        "iou": jaccard.compute().item(),
        "dice": sum(all_dice_scores) / len(all_dice_scores) if all_dice_scores else 0.0,
        "accuracy": accuracy_metric.compute().item(),
        "f1": f1_metric.compute().item(),
        "precision": precision_metric.compute().item(),
        "recall": recall_metric.compute().item(),
        "inference_time": sum(inference_times) / len(inference_times) if inference_times else 0.0
    }
    jaccard.reset(); accuracy_metric.reset(); f1_metric.reset(); precision_metric.reset(); recall_metric.reset()
    return metrics

In [ ]:
# =================================================================================
# PHẦN VI. MAIN WORKFLOW
# =================================================================================

def run_multitask_pipeline(config, train_list, test_list, image_transform, run_id):
    """Quy trình huấn luyện và đánh giá cho mô hình Multitask."""
    print(f"\n===== BẮT ĐẦU QUY TRÌNH MULTITASK (RUN {run_id+1}) =====")
    
    # --- Data & Transforms ---
    mask_transform = transforms.Compose([transforms.ToTensor()])
    train_joint_transform = Compose([Resize(config.IMG_SIZE), RandomHorizontallyFlip(), RandomVerticallyFlip(), RandomRotate(15)])
    test_joint_transform = Compose([Resize(config.IMG_SIZE)])

    train_dataset = BUSIDataset(train_list, train_joint_transform, image_transform, mask_transform)
    test_dataset = BUSIDataset(test_list, test_joint_transform, image_transform, mask_transform)
    train_loader = data.DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
    test_loader = data.DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
    print("Đã tạo DataLoader cho Multitask.\n")
    
    # --- Model, Loss, Optimizer ---
    model = UNet_MTL(config).to(config.DEVICE)
    seg_criterion = DiceLoss()
    cls_criterion = MarginLoss() if config.CLASSIFICATION_HEAD == 'capsnet' else nn.CrossEntropyLoss()
    combined_loss_func = UncertaintyWeightedLoss().to(config.DEVICE)
    optimizer = torch.optim.Adam(list(model.parameters()) + list(combined_loss_func.parameters()), lr=config.LEARNING_RATE)
    
    early_stopper = EarlyStopping(patience=config.EARLY_STOPPING_PATIENCE, path=f"{config.MTL_MODEL_CHECKPOINT_PATH}_run{run_id+1}.pt")
    
    # --- Training Loop ---
    print("===== BẮT ĐẦU HUẤN LUYỆN MULTITASK =====")
    for epoch in range(config.EPOCHS):
        if config.TEST_IMPLEMENTATION and epoch >= 15: break
        train_loss, _, _ = train_one_epoch(model, train_loader, optimizer, seg_criterion, cls_criterion, combined_loss_func, epoch)
        eval_metrics = evaluate(model, test_loader, seg_criterion, cls_criterion, config)
        val_avg = (eval_metrics['dice']+eval_metrics['f1'])/2
        print(f"Epoch {epoch+1}/{config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Dice: {eval_metrics['dice']:.4f} | Val F1: {eval_metrics['f1']:.4f}")

        early_stopper(val_avg, model)
        if early_stopper.early_stop:
            print("Dừng sớm!")
            break

    # --- Final Evaluation ---
    print("\n===== ĐÁNH GIÁ CUỐI CÙNG (MULTITASK) =====")
    model.load_state_dict(torch.load(early_stopper.path))
    final_metrics = evaluate(model, test_loader, seg_criterion, cls_criterion, config)
    return final_metrics, model

def run_sequential_pipeline(config, train_list, test_list, image_transform, run_id):
    """Quy trình huấn luyện và đánh giá cho mô hình Sequential."""
    print(f"\n===== BẮT ĐẦU QUY TRÌNH SEQUENTIAL (RUN {run_id+1}) =====")

    # --- Common Data & Transforms ---
    mask_transform = transforms.Compose([transforms.ToTensor()])
    train_joint_transform = Compose([Resize(config.IMG_SIZE), RandomHorizontallyFlip(), RandomVerticallyFlip(), RandomRotate(15)])
    test_joint_transform = Compose([Resize(config.IMG_SIZE)])

    # ===========================================================
    # STAGE 1: TRAIN SEGMENTATION MODEL
    # ===========================================================
    print("\n--- STAGE 1: HUẤN LUYỆN MÔ HÌNH SEGMENTATION ---")
    seg_model = UNet_Segmentation(config).to(config.DEVICE)
    seg_criterion = DiceLoss()
    optimizer_seg = torch.optim.Adam(seg_model.parameters(), lr=config.LEARNING_RATE)
    early_stopper_seg = EarlyStopping(patience=config.EARLY_STOPPING_PATIENCE, path=f"{config.SEG_MODEL_CHECKPOINT_PATH}_run{run_id+1}.pt")
    
    train_dataset_seg = BUSIDataset(train_list, train_joint_transform, image_transform, mask_transform)
    test_dataset_seg = BUSIDataset(test_list, test_joint_transform, image_transform, mask_transform)
    train_loader_seg = data.DataLoader(train_dataset_seg, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=0)
    test_loader_seg = data.DataLoader(test_dataset_seg, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0)

    for epoch in range(int(config.EPOCHS/2)):
        if config.TEST_IMPLEMENTATION and epoch >= 15: break
        train_loss = train_seg_only(seg_model, train_loader_seg, optimizer_seg, seg_criterion, epoch)
        val_loss, val_dice = evaluate_seg_only(seg_model, test_loader_seg, seg_criterion)
        print(f"Epoch {epoch+1} [Seg] | Train Loss: {train_loss:.4f} | Val Dice: {val_dice:.4f}")
        early_stopper_seg(val_dice, seg_model)
        if early_stopper_seg.early_stop:
            print("Dừng sớm Stage 1!")
            break
            
    # Load best segmentation model
    best_seg_model = UNet_Segmentation(config).to(config.DEVICE)
    best_seg_model.load_state_dict(torch.load(early_stopper_seg.path))
    best_seg_model.eval()
    print("Đã tải mô hình segmentation tốt nhất.")

    # ===========================================================
    # STAGE 2: TRAIN CLASSIFICATION MODEL
    # ===========================================================
    print("\n--- STAGE 2: HUẤN LUYỆN MÔ HÌNH CLASSIFICATION ---")
    cls_model = ClassificationModel(config).to(config.DEVICE)
    cls_criterion = MarginLoss() if config.CLASSIFICATION_HEAD == 'capsnet' else nn.CrossEntropyLoss()
    optimizer_cls = torch.optim.Adam(cls_model.parameters(), lr=config.LEARNING_RATE)
    early_stopper_cls = EarlyStopping(patience=config.EARLY_STOPPING_PATIENCE, path=f"{config.CLS_MODEL_CHECKPOINT_PATH}_run{run_id+1}.pt", mode='max') 

    # Sử dụng ClassificationSequentialDataset với image_transform đã có
    # Class này sẽ tự động đọc ảnh, áp dụng transform và đưa qua seg_model
    train_dataset_cls = ClassificationSequentialDataset(train_list, best_seg_model, config.IMG_SIZE, image_transform=image_transform)
    test_dataset_cls = ClassificationSequentialDataset(test_list, best_seg_model, config.IMG_SIZE,image_transform=image_transform)
    train_loader_cls = data.DataLoader(train_dataset_cls, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=0)
    test_loader_cls = data.DataLoader(test_dataset_cls, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0)

    for epoch in range(int(config.EPOCHS/2)):
        if config.TEST_IMPLEMENTATION and epoch >= 30: break
        train_loss = train_cls_only(cls_model, train_loader_cls, optimizer_cls, cls_criterion, epoch)
        eval_metrics_cls = evaluate_cls_only(cls_model, test_loader_cls, cls_criterion)
        val_f1 = eval_metrics_cls['f1']
        print(f"Epoch {epoch+1} [Cls] | Train Loss: {train_loss:.4f} | Val F1: {val_f1:.4f}")
        early_stopper_cls(val_f1, cls_model)
        if early_stopper_cls.early_stop:
            print("Dừng sớm Stage 2!")
            break

    # Load best classification model
    best_cls_model = ClassificationModel(config).to(config.DEVICE)
    best_cls_model.load_state_dict(torch.load(early_stopper_cls.path))
    best_cls_model.eval()
    print("Đã tải mô hình classification tốt nhất.")

    # --- Final Evaluation ---
    print("\n===== ĐÁNH GIÁ CUỐI CÙNG (SEQUENTIAL) =====")
    # Dùng test_loader_seg vì nó chứa (ảnh gốc, mask gốc, label)
    final_metrics = evaluate_sequential_pipeline(best_seg_model, best_cls_model, test_loader_seg, seg_criterion)
    return final_metrics, (best_seg_model, best_cls_model)


if __name__ == '__main__':
    mp.set_start_method('spawn', force=True)
    all_runs_metrics = []
    
    for run in range(config.NUM_RUNS):
        print(f"\n{'='*30} BẮT ĐẦU LẦN CHẠY {run+1}/{config.NUM_RUNS} {'='*30}")
        current_seed = config.SEED + run
        set_seed(current_seed)
        print(f"Cố định seed cho lần chạy này là: {current_seed}")

        print("\n===== CẤU HÌNH THỬ NGHIỆM =====")
        print(f"Device: {config.DEVICE}")
        print(f"Model Type: {config.MODEL_TYPE}")
        print(f"Backbone: {config.BACKBONE}")
        print(f"Sử dụng Deformable Conv: {config.USE_Deform}")
        print(f"Nhánh phân loại: {config.CLASSIFICATION_HEAD}")
        print(f"Chế độ Debug: {config.TEST_IMPLEMENTATION}\n")

        # --- Bước 1: Load và chia dataset ---
        full_dataset_list = make_dataset(config.DATASET_PATH, config.OUTPUT_MASK_DIR)
        labels_for_split = [item[2] for item in full_dataset_list]
        train_list, test_list = train_test_split(
            full_dataset_list, test_size=config.TEST_RATIO, random_state=config.SEED, stratify=labels_for_split
        )
        print(f"Dataset được chia thành: {len(train_list)} mẫu train và {len(test_list)} mẫu test.")

        # --- Bước 2: Tính Mean/Std và tạo Transform chung ---
        # Chỉ cần transform cơ bản để tính toán
        temp_transform = Compose([Resize(config.IMG_SIZE)])
        temp_dataset = BUSIDataset(train_list, joint_transform=temp_transform, image_transform=transforms.ToTensor())
        mean, std = calculate_mean_std(temp_dataset)
        
        image_final_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])

        print(f"Calculated Mean: {mean}\nCalculated Std: {std}\n")
        
        # --- Bước 3: Chạy pipeline tương ứng ---
        if config.MODEL_TYPE == 'multitask':
            final_metrics, model = run_multitask_pipeline(config, train_list, test_list, image_final_transform, run)
        elif config.MODEL_TYPE == 'sequential':
            final_metrics, models = run_sequential_pipeline(config, train_list, test_list, image_final_transform, run)
            model_seg = models[0] # Lấy seg_model để tính FLOPs/params
            model_cls = models[1] # Lấy cls_model để tính FLOPs/params
        else:
            raise ValueError(f"Model type '{config.MODEL_TYPE}' không được hỗ trợ.")

        all_runs_metrics.append(final_metrics)

        # --- Bước 4: Tính FLOPs và Params (chỉ lần đầu) ---
        if run == 0:
            if config.MODEL_TYPE=='multitask':
                dummy_input = torch.randn(1, 3, config.IMG_SIZE, config.IMG_SIZE).to(config.DEVICE)           
                flops, params = profile(model, inputs=(dummy_input,), verbose=False)
                print(f"\n===== Hiệu suất tính toán ({'MTL'}) =====")
                print(f"Parameters: {params / 1e6:.2f}M")
                print(f"FLOPs: {flops / 1e9:.2f}G\n")
            else:
                # Dummy input cho model segmentation (3 kênh)
                dummy_input_seg = torch.randn(1, 3, config.IMG_SIZE, config.IMG_SIZE).to(config.DEVICE)
                flops_seg, params_seg = profile(model_seg, inputs=(dummy_input_seg,), verbose=False)
            
                # Dummy input cho model classification (1 kênh)
                dummy_input_cls = torch.randn(1, 1, config.IMG_SIZE, config.IMG_SIZE).to(config.DEVICE)
                flops_cls, params_cls = profile(model_cls, inputs=(dummy_input_cls,), verbose=False)
            
                # Tổng hợp lại
                flops = flops_seg + flops_cls
                params = params_seg + params_cls
                print(f"===== Hiệu suất tính toán (Sequential) =====")
                print(f"Parameters: {params / 1e6:.2f}M")
                print(f"FLOPs: {flops / 1e9:.2f}G\n")
            
        # --- Bước 5: In kết quả lần chạy hiện tại ---
        print(f"--- Kết quả lần chạy {run+1} ---")
        for key, value in final_metrics.items():
            print(f"{key.capitalize():<15}: {value:.4f}")
        print("-" * 40)

    # --- Bước 6: Tổng hợp và báo cáo kết quả cuối cùng ---
    if all_runs_metrics:
        print(f"\n\n{'='*30} KẾT QUẢ TỔNG HỢP SAU {config.NUM_RUNS} LẦN CHẠY {'='*30}")
        results_df = pd.DataFrame(all_runs_metrics)
        mean_metrics = results_df.mean()
        std_metrics = results_df.std()

        print(f"Cấu hình: Model '{config.MODEL_TYPE}', Backbone '{config.BACKBONE}', Phân loại '{config.CLASSIFICATION_HEAD}', Deformable: {config.USE_Deform}")
        print("-" * 60)
        print(f"{'Metric':<20} | {'Mean':>15} | {'Std Dev':>15}")
        print("-" * 60)
        for metric_name in mean_metrics.index:
            mean_val = mean_metrics[metric_name]
            std_val = std_metrics[metric_name]
            print(f"{metric_name.capitalize():<20} | {mean_val:>15.4f} | {std_val:>15.4f}")
        print("-" * 60)
        results_df.to_csv('output.csv', index=False)
        print("Đã lưu kết quả vào output.csv")